In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, GRU, Reshape, Input, Lambda
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras import backend as K


In [2]:

# Parameters
seq_length = 10  # Length of word sequence
vocab_size = 5000  # Vocabulary size


# # Loading data from 'tweets.csv'
# data = pd.read_csv('tweets.csv')
# real_tweets = data['tweet'].values


# test tweets
real_tweets = [
    "The weather is nice today",
    "I love learning new things",
    "GANs are fascinating",
    "This is a beautiful day",
    "Machine learning is fun",
    "I hate learning old things",
    "CNNs are beautiful",
    "This is a sad code",
    "deep learning is bad"
]


# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(real_tweets)

real_sequences = tokenizer.texts_to_sequences(real_tweets)

padded_sequences = pad_sequences(real_sequences, maxlen=seq_length, padding='post', truncating='post')

one_hot_sequences = np.array([to_categorical(seq, num_classes=vocab_size) for seq in padded_sequences])

print(f'Shape of one_hot_sequences: {padded_sequences.shape}')

print(padded_sequences.shape,'\n',padded_sequences)

Shape of one_hot_sequences: (9, 10)
(9, 10) 
 [[ 9 10  1 11 12  0  0  0  0  0]
 [ 3 13  2 14  4  0  0  0  0  0]
 [15  5 16  0  0  0  0  0  0  0]
 [ 6  1  7  8 17  0  0  0  0  0]
 [18  2  1 19  0  0  0  0  0  0]
 [ 3 20  2 21  4  0  0  0  0  0]
 [22  5  8  0  0  0  0  0  0  0]
 [ 6  1  7 23 24  0  0  0  0  0]
 [25  2  1 26  0  0  0  0  0  0]]


In [3]:
# Gumbel-Softmax sampling functions
def sample_gumbel(shape, eps=1e-20):
    """Sample from Gumbel(0, 1) distribution."""
    U = tf.random.uniform(shape, minval=0, maxval=1)
    return -K.log(-K.log(U + eps) + eps)

def gumbel_softmax(logits, temperature):
    """Sample from the Gumbel-Softmax distribution."""
    gumbel_noise = sample_gumbel(tf.shape(logits))
    y = logits + gumbel_noise
    return K.softmax(y / temperature)

def gumbel_softmax_sample(logits, temperature):
    """Sample from Gumbel-Softmax and convert to word indices."""
    gumbel_softmax_output = gumbel_softmax(logits, temperature)
    # Convert softmax output to word indices
    word_indices = tf.argmax(gumbel_softmax_output, axis=-1)
    return word_indices

# Generator model
def build_generator(vocab_size, seq_length):
    model = Sequential(name='generator')
    model.add(Dense(120, input_dim=100))  # Random noise vector length is 100
    model.add(Reshape((seq_length, 12)))  # Reshape to (sequence_length, features)
    model.add(GRU(128, return_sequences=True))  # Recurrent layer for sequence generation
    model.add(Dense(vocab_size))  # Output logits for vocab distribution
    # Use Gumbel-Softmax to generate a sequence of word indices
    model.add(Lambda(lambda x: gumbel_softmax_sample(x, temperature=0.5)))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0002, 0.5))
    model.summary()
    return model

# Discriminator model
def build_discriminator(vocab_size, seq_length):
    model = Sequential(name='discriminator')
    model.add(Input(shape=(seq_length,)))  # Expecting a sequence of word indices
    model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length))  # Embedding layer
    model.add(LSTM(128))  # Recurrent layer for processing sequences
    model.add(Dense(1, activation='sigmoid'))  # Binary classification: real or fake
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    model.summary()
    return model

# GAN model: Combined Generator and Discriminator
def build_gan(generator, discriminator):
    discriminator.trainable = False  # We freeze the discriminator when training the generator
    model = Sequential(name='GAN')
    model.add(generator)
    # model.add(Reshape((10,)))
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    print(model.summary())
    return model

In [4]:
# Instantiate the models
generator = build_generator(vocab_size, seq_length)
discriminator = build_discriminator(vocab_size, seq_length)
gan = build_gan(generator, discriminator)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 120)                 │          12,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 10, 12)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 10, 128)             │          54,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10, 5000)            │         645,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 711,648 (2.71 MB)

 Trainable params: 711,648 (2.71 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ generator (Sequential)               │ (None, 10)                  │         711,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ discriminator (Sequential)           │ (None, 1)                   │         341,777 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,053,425 (4.02 MB)

 Trainable params: 711,648 (2.71 MB)

 Non-trainable params: 341,777 (1.30 MB)

None


In [5]:
# Generate fake data (fake word sequences)
def generate_fake_samples(generator, batch_size):
    noise = np.random.normal(0, 1, (batch_size, 100))  # Generate random noise
    # print('noise:',noise)
    generated_sequences = generator.predict(noise)
    print('generated_sequences = generator.predict(noise):',generated_sequences.shape,'\n',generated_sequences)

    # Convert softmax output to word indices (sampled from the probability distribution)
    word_indices = generated_sequences
    # word_indices = np.argmax(generated_sequences, axis=-1)
    print('word_indices = np.argmax(generated_sequences, axis=-1):',word_indices.shape)
    return word_indices

In [6]:
noise = np.random.normal(0, 1, (32, 100))
noise.shape

(32, 100)

In [7]:
# Training loop (simplified)
def train_gan(generator, discriminator, gan, real_data, epochs=1, batch_size=32):
    for epoch in range(epochs):
        # Train Discriminator
        real_seqs = real_data[np.random.randint(0, real_data.shape[0], batch_size)]
        print('Real sequences shape (before one-hot encoding):', real_seqs.shape)

        # Ensure real sequences are one-hot encoded (convert (batch_size, 10) -> (batch_size, 10, 5000))
        real_seqs_one_hot = np.array([to_categorical(seq, num_classes=vocab_size) for seq in real_seqs])
        print('Real sequences shape (after one-hot encoding):', real_seqs_one_hot.shape)

        # Generate fake sequences from the generator
        fake_seqs = generate_fake_samples(generator, batch_size)
        print('Fake sequences shape:', fake_seqs.shape)
        # Labels for real (1) and fake (0)
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        # Train on real and fake sequences
        d_loss_real = discriminator.train_on_batch(real_seqs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_seqs, fake_labels)

        # Train Generator via GAN
        noise = np.random.normal(0, 1, (batch_size, 100))  # Generate noise for GAN training
        valid_labels = np.ones((batch_size, 1))  # We want the generator to trick the discriminator
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print losses at intervals
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: D loss real: {d_loss_real}, D loss fake: {d_loss_fake}, G loss: {g_loss}")

In [8]:
# Prepare real data
real_data = padded_sequences
print('real_data:',real_data.shape)
# Train GAN
train_gan(generator, discriminator, gan, real_data, batch_size=6)


real_data: (9, 10)
Real sequences shape (before one-hot encoding): (6, 10)
Real sequences shape (after one-hot encoding): (6, 10, 5000)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
generated_sequences = generator.predict(noise): (6, 10) 
 [[4410 3908 2802 2739 1497 1192 3513  978 2224 1139]
 [3085 1240 2801 3230 1000 3808 2146 2991 1825 2962]
 [ 659 4063 1084 2620 2021 3624  868 3952 2550 4897]
 [1700  693  330 3127 3657  343 3170 2499 3642  636]
 [4178 3969  260 1173 2918 2565 2863 2932 2174 2686]
 [1439 3269 4612 3064  195 1800 2291 1021 1261 4722]]
word_indices = np.argmax(generated_sequences, axis=-1): (6, 10)
Fake sequences shape: (6, 10)


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


ValueError: No gradients provided for any variable.

In [9]:
print(real_data.shape)
print(real_data.shape[0])
x=np.random.randint(0, 9, 14)
print('np.random.randint(0, 9, 14):',x)
real_seq = real_data[x]
print(real_seq.shape)
real_seq

(9, 10)
9
np.random.randint(0, 9, 14): [3 7 2 7 4 8 6 5 1 8 1 5 5 4]
(14, 10)


array([[ 6,  1,  7,  8, 17,  0,  0,  0,  0,  0],
       [ 6,  1,  7, 23, 24,  0,  0,  0,  0,  0],
       [15,  5, 16,  0,  0,  0,  0,  0,  0,  0],
       [ 6,  1,  7, 23, 24,  0,  0,  0,  0,  0],
       [18,  2,  1, 19,  0,  0,  0,  0,  0,  0],
       [25,  2,  1, 26,  0,  0,  0,  0,  0,  0],
       [22,  5,  8,  0,  0,  0,  0,  0,  0,  0],
       [ 3, 20,  2, 21,  4,  0,  0,  0,  0,  0],
       [ 3, 13,  2, 14,  4,  0,  0,  0,  0,  0],
       [25,  2,  1, 26,  0,  0,  0,  0,  0,  0],
       [ 3, 13,  2, 14,  4,  0,  0,  0,  0,  0],
       [ 3, 20,  2, 21,  4,  0,  0,  0,  0,  0],
       [ 3, 20,  2, 21,  4,  0,  0,  0,  0,  0],
       [18,  2,  1, 19,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [10]:
discrtest = build_discriminator(vocab_size,seq_length)
gentest = build_generator(vocab_size,seq_length)
discrtest(real_seq)

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 10, 50)              │         250,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │          91,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 341,777 (1.30 MB)

 Trainable params: 341,777 (1.30 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 120)                 │          12,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 10, 12)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 10, 128)             │          54,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10, 5000)            │         645,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_1 (Lambda)                    │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 711,648 (2.71 MB)

 Trainable params: 711,648 (2.71 MB)

 Non-trainable params: 0 (0.00 B)

<tf.Tensor: shape=(14, 1), dtype=float32, numpy=
array([[0.49419534],
       [0.49314162],
       [0.49235117],
       [0.49314162],
       [0.49343693],
       [0.4930071 ],
       [0.49296173],
       [0.4936419 ],
       [0.49314418],
       [0.4930071 ],
       [0.49314418],
       [0.49364188],
       [0.49364188],
       [0.49343693]], dtype=float32)>

In [11]:
fake_seqs = generate_fake_samples(gentest, 3)
print('fake_seqs:',fake_seqs.shape,'\n',fake_seqs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
generated_sequences = generator.predict(noise): (3, 10) 
 [[ 837 4195 1538 3897 3225 4793 2136 2896  720 2816]
 [4669 4181 2553 2424 1081 1605 1072  845 4811 4840]
 [3805 1921 1772 1672 4294 4804 3664 3931 3474 1375]]
word_indices = np.argmax(generated_sequences, axis=-1): (3, 10)
fake_seqs: (3, 10) 
 [[ 837 4195 1538 3897 3225 4793 2136 2896  720 2816]
 [4669 4181 2553 2424 1081 1605 1072  845 4811 4840]
 [3805 1921 1772 1672 4294 4804 3664 3931 3474 1375]]
